# Hands-on data engineering Vantage AI

Deze sessie gaan we neurale netwerken trainen om simpele images te classificeren van de CIFAR-10 dataset. 

## Dependency management
Deze notebook gaat er vanuit dat je de volgende python dependencies geïnstalleerd hebt:
- Jupyter
- Tensorflow
- Keras
- Matplotlib
- SKLearn

Opdracht: _Schrijf een `requirements.txt` waarmee de requirements van deze notebook makkelijk geïnstalleerd kunnen worden._

## Data inladen

De data bestaat uit 3 delen: train, validatie en test set.

Opdracht: _Er is veel herhaling in deze code. Splits dit op in leesbare en herbruikbare code. Denk hierbij aan de engineering principes die we hebben behandeld._

In [2]:
import numpy as np
import os
import tarfile
from urllib.request import urlretrieve
import pickle
import random


def load_data(verbose=True):
    dataset_dir_base = _get_dataset_dir_base_path()
    dataset_dir = _get_dataset_dir_path()
    download_data(dataset_dir_base, dataset_dir)
    
    no_of_batches = 4
    no_of_samples = 10000
    train_X, train_y = get_train_sets(dataset_dir, no_of_batches, no_of_samples)
    val_X, val_y = get_validation_set(dataset_dir)
    test_X, test_y = get_test_set(dataset_dir)
    label_to_names = get_label_to_names(dataset_dir)
    
    if verbose:
        print("training set size: data = {}, labels = {}".format(train_X.shape, train_y.shape))
        print("validation set size: data = {}, labels = {}".format(val_X.shape, val_y.shape))

        print("Test set size: data = "+str(test_X.shape)+", labels = "+str(test_y.shape))
    
    return train_X, train_y, val_X, val_y, test_X, test_y, label_to_names

def _get_dataset_dir_base_path():
    return os.path.join(os.getcwd(), "..", "data", "raw")

def _get_dataset_dir_path():
    dataset_dir_base = _get_dataset_dir_base_path()
    return os.path.join(dataset_dir_base, "cifar-10-batches-py")

def download_data(output_dir, output_path):    
    if not os.path.exists(output_path):
        print("Downloading data...")
        urlretrieve("http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", os.path.join(output_dir, "cifar-10-python.tar.gz"))
        tar = tarfile.open(os.path.join(output_dir, "cifar-10-python.tar.gz"))
        tar.extractall(output_dir)
        tar.close()
    
def get_train_sets(dataset_dir, no_of_batches, n_samples):
    sample_size = no_of_batches*n_samples
    train_X = np.zeros((sample_size, 3, 32, 32), dtype="float32")
    train_y = np.zeros((sample_size, 1), dtype="ubyte").flatten()
    
    for i in range(0, no_of_batches):
        cifar_batch = _get_cifar_dict(dataset_dir, dict_name="data_batch_"+str(i+1))
        train_X[i*n_samples:(i+1)*n_samples] = (cifar_batch['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
        train_y[i*n_samples:(i+1)*n_samples] = np.array(cifar_batch['labels'], dtype='ubyte')
        
    return train_X, train_y

def get_validation_set(dataset_dir):
    cifar_batch_5 = _get_cifar_dict(dataset_dir, dict_name="data_batch_5")
    val_X = (cifar_batch_5['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
    val_y = np.array(cifar_batch_5['labels'], dtype='ubyte')
    return val_X, val_y

def get_label_to_names(dataset_dir):
    cifar_dict = _get_cifar_dict(dataset_dir, dict_name="batches.meta")
    label_to_names = {k:v for k, v in zip(range(10), cifar_dict['label_names'])}
    return label_to_names

def get_test_set(dataset_dir):
    cifar_test = _get_cifar_dict(dataset_dir, dict_name="test_batch")

    test_X = (cifar_test['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
    test_y = np.array(cifar_test['labels'], dtype='ubyte')
    return test_X, test_y

def _get_cifar_dict(dataset_dir, dict_name):
    with open(os.path.join(dataset_dir, dict_name), "rb") as f:
        cifar_dict = pickle.load(f, encoding="latin1")
    return cifar_dict

### Preprocessing
Bij CIFAR10 is er niet veel preprocessing nodig. Normalisatie van de data is vaak een goed idee, vantevoren berekenen we de gemiddelde pixelwaarde en bij het batchgewijs trainen normaliseren we de data aan de hand van die waarde. Het is een goed idee om deze mean in een pickle bestand op te slaan, en die dan in te laden bij het opstarten zodat voor predicten niet de hele dataset nodig is. 

In [12]:
# PREPROCESS


train_X, train_y, val_X, val_y, test_X, test_y, label_to_names = load_data()

# Conv nets trainen duurt erg lang op CPU, dus we gebruiken maar een klein deel
# van de data nu, als er tijd over is kan je proberen je netwerk op de volledige set te runnen
train_X = train_X[:10000]
train_y = train_y[:10000]

def calc_mean_std(X):
    mean = np.mean(X)
    std = np.std(X)
    return mean, std

def normalize(data, mean, std):
    return (data-mean)/std

#De data van train_X is genoeg om de mean en std van de hele set nauwkeurig te benaderen
mean,std = calc_mean_std(train_X)
test_X = normalize(test_X,mean,std)
val_X = normalize(val_X,mean,std)
train_X = normalize(train_X ,mean,std)


training set size: data = (40000, 3, 32, 32), labels = (40000,)
validation set size: data = (10000, 3, 32, 32), labels = (10000,)
Test set size: data = (10000, 3, 32, 32), labels = (10000,)


In [ ]:
train_X.shape

# Definieer model
We gebruiken de volledige images om een convolutioneel neuraal netwerk te definieren en te trainen. Alhoewel de data science niet de focus heeft in deze cursus is het wel belangrijk om te begrijpen wat er gebeurt, dus schroom niet ook vragen te stellen over het model.

In [6]:
from keras.models import Model
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, Input
from sklearn.metrics import classification_report

def conv_net():
    # We definieren de input van het netwerk als de shape van de input,
    # minus de dimensie van het aantal plaatjes, uiteindelijk dus (3, 32, 32).
    input = Input(shape=train_X.shape[1:])
    
    # Eerste convolutielaag
    # Padding valid betekent dat we enkel volledige convoluties gebruiken, zonder padding
    # Data format channels_first betekent dat de channels eerst komen, en dan pas de size van ons plaatje
    # Dus (3, 32, 32) in plaats van (32, 32, 3)
    conv = Conv2D(filters=16, kernel_size=(3,3), padding='valid',
                  data_format='channels_first', activation='relu')(input)
    
    # Nog een convolutielaag, dit keer met stride=2 om de inputsize te verkleinen
    conv = Conv2D(filters=32, kernel_size=(3,3), padding='valid',
                  data_format='channels_first', activation='relu', strides=(2, 2))(conv)
    
    #Voeg een flatten laag toe, om te schakelen naar de dense layer
    flatten = Flatten()(conv)
   
    # De softmax laag voor de probabilities 
    nr_classes=10
    output_layer = Dense(units=nr_classes, activation='softmax')(flatten)
    
    model = Model(inputs=input, outputs=output_layer)
    
    # Het model moet nog gecompiled worden en loss+learning functie gespecificeerd worden
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


model = conv_net()

model.fit(x=train_X, y=train_y, batch_size=50, epochs=1, validation_data=(val_X, val_y), verbose=2)


Train on 10000 samples, validate on 10000 samples
Epoch 1/1
 - 39s - loss: 1.7260 - acc: 0.3787 - val_loss: 1.4962 - val_acc: 0.4775


In [14]:
# PREDICT
predictions = np.array(model.predict(test_X, batch_size=100))
# test_y = np.array(test_y, dtype=np.int32)
#Take the highest prediction
predictions = np.argmax(predictions, axis=1)

#Print resultaten
print("Accuracy = {}".format(np.sum(predictions == test_y) / float(len(predictions))))
print(classification_report(test_y, predictions, target_names=list(label_to_names.values())))

Accuracy = 0.4834
             precision    recall  f1-score   support

   airplane       0.60      0.46      0.52      1000
 automobile       0.63      0.58      0.60      1000
       bird       0.38      0.41      0.40      1000
        cat       0.40      0.32      0.36      1000
       deer       0.45      0.26      0.33      1000
        dog       0.45      0.32      0.37      1000
       frog       0.46      0.66      0.54      1000
      horse       0.41      0.67      0.51      1000
       ship       0.57      0.64      0.60      1000
      truck       0.54      0.52      0.53      1000

avg / total       0.49      0.48      0.48     10000



In [13]:
# PREDICT
predictions = np.array(model.predict(test_X, batch_size=100))
# test_y = np.array(test_y, dtype=np.int32)
#Take the highest prediction
predictions = np.argmax(predictions, axis=1)

#Print resultaten
print("Accuracy = {}".format(np.sum(predictions == test_y) / float(len(predictions))))
print(classification_report(test_y, predictions, target_names=list(label_to_names.values())))

Accuracy = 0.4834
             precision    recall  f1-score   support

   airplane       0.60      0.46      0.52      1000
 automobile       0.63      0.58      0.60      1000
       bird       0.38      0.41      0.40      1000
        cat       0.40      0.32      0.36      1000
       deer       0.45      0.26      0.33      1000
        dog       0.45      0.32      0.37      1000
       frog       0.46      0.66      0.54      1000
      horse       0.41      0.67      0.51      1000
       ship       0.57      0.64      0.60      1000
      truck       0.54      0.52      0.53      1000

avg / total       0.49      0.48      0.48     10000



In [11]:
classification_report(test_y, predictions, target_names=list(label_to_names.values()), output_dict=True)

TypeError: classification_report() got an unexpected keyword argument 'output_dict'

In [ ]:
model.summary()

## Opdracht
Wat we graag willen is deze notebook uitgewerkt in een package met het cookiecutter template. We willen dan graag een splitsing tussen het trainingsdeel en het scoring deel. 

Het trainingsdeel levert een model en eventuele metadata op (opgeslagen op disk).  
Het scoringsdeel gebruikt het model om de testset te predicten.  
Runnen met: `python <filename> 'scoring'` of `python <filename> 'training'`

Gebruik goede error handling voor bijvoorbeeld het predicten zonder model, of een verkeerd argument meegeven.

In [17]:
np.__version__

'1.13.3'

In [20]:
os.listdir('../models')

['.gitkeep', 'finalized_model.pickle']

In [22]:
model = pickle.load(open('../models/finalized_model.pickle', 'rb'))
model

In [23]:
type(model)

NoneType